In [1]:
import pandas as pd
import numpy as np

In [2]:
# 데이터 로드
df = pd.read_csv('./online_retail_clean.csv')
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [3]:
# 고객별 구매 횟수
customer_purchases = df.groupby('CustomerID').agg({
    'InvoiceNo': 'nunique',  # 구매 횟수
    'Revenue': 'sum',         # 총 구매 금액
    'InvoiceDate': ['min', 'max']  # 첫 구매, 마지막 구매
}).reset_index()

customer_purchases.columns = ['CustomerID', 'PurchaseCount', 'TotalRevenue', 'FirstPurchase', 'LastPurchase']

In [4]:
# 재구매 여부
customer_purchases['IsRepeat'] = customer_purchases['PurchaseCount'] > 1

In [5]:
# 재구매까지 걸린 기간 (일)
customer_purchases['DaysToRepurchase'] = (
    customer_purchases['LastPurchase'] - customer_purchases['FirstPurchase']
).dt.days

In [6]:
# 재구매 고객만 필터
repeat_customers = customer_purchases[customer_purchases['IsRepeat']]

In [7]:
print(f"\n전체 고객 수: {len(customer_purchases):,}명")
print(f"재구매 고객: {len(repeat_customers):,}명 ({len(repeat_customers)/len(customer_purchases)*100:.2f}%)")
print(f"일회성 고객: {len(customer_purchases) - len(repeat_customers):,}명")


전체 고객 수: 4,338명
재구매 고객: 2,845명 (65.58%)
일회성 고객: 1,493명


In [8]:
print(f"\n평균 구매 횟수:")
print(f"  전체: {customer_purchases['PurchaseCount'].mean():.2f}회")
print(f"  재구매 고객: {repeat_customers['PurchaseCount'].mean():.2f}회")


평균 구매 횟수:
  전체: 4.27회
  재구매 고객: 5.99회


In [9]:
print(f"\n평균 구매 금액:")
print(f"  전체: £{customer_purchases['TotalRevenue'].mean():.2f}")
print(f"  재구매 고객: £{repeat_customers['TotalRevenue'].mean():.2f}")
print(f"  일회성 고객: £{customer_purchases[~customer_purchases['IsRepeat']]['TotalRevenue'].mean():.2f}")


평균 구매 금액:
  전체: £2054.27
  재구매 고객: £2915.68
  일회성 고객: £412.80


In [10]:
print(f"\n재구매까지 평균 소요 기간:")
print(f"  평균: {repeat_customers['DaysToRepurchase'].mean():.1f}일")
print(f"  중앙값: {repeat_customers['DaysToRepurchase'].median():.1f}일")


재구매까지 평균 소요 기간:
  평균: 198.9일
  중앙값: 207.0일


In [12]:
# 첫 구매 금액 분석
first_purchase_amount = df.groupby('CustomerID').first()
first_purchase_amount = df.sort_values('InvoiceDate').groupby('CustomerID').first()['Revenue']

customer_purchases['FirstPurchaseAmount'] = customer_purchases['CustomerID'].map(first_purchase_amount)

print(f"\n첫 구매 금액:")
print(f"  재구매 고객: £{customer_purchases[customer_purchases['IsRepeat']]['FirstPurchaseAmount'].mean():.2f}")
print(f"  일회성 고객: £{customer_purchases[~customer_purchases['IsRepeat']]['FirstPurchaseAmount'].mean():.2f}")



첫 구매 금액:
  재구매 고객: £37.09
  일회성 고객: £88.70


In [13]:
# 재구매 배율
revenue_multiplier = (
    customer_purchases[customer_purchases['IsRepeat']]['TotalRevenue'].mean() /
    customer_purchases[~customer_purchases['IsRepeat']]['TotalRevenue'].mean()
)
print(f"\n재구매 고객의 총 구매 금액 배율: {revenue_multiplier:.2f}배")


재구매 고객의 총 구매 금액 배율: 7.06배


In [14]:
# 저장
customer_purchases.to_csv('./customer_analysis.csv', index=False)
print("\n✅ 고객 분석 완료: customer_analysis.csv")


✅ 고객 분석 완료: customer_analysis.csv


# 재구매 고객 vs 일회성 고객 비교

전체 고객 수: 4,338명    
재구매 고객: 2,845명 (65.58%)    
일회성 고객: 1,493명    

**평균 구매 횟수**:  
  전체: 4.27회    
  재구매 고객: 5.99회    

**평균 구매 금액**:    
  전체: £2054.27    
  재구매 고객: £2915.68    
  일회성 고객: £412.80    

**재구매까지 평균 소요 기간**:    
  평균: 198.9일    
  중앙값: 207.0일    

**첫 구매 금액**:    
  재구매 고객: £37.09    
  일회성 고객: £88.70    

재구매 고객의 총 구매 금액 배율: 7.06배


# 핵심 인사이트
- 재구매율: 65.58% (매우 높음)   
- 재구매 고객의 가치: 일회성 대비 7.06배   

- 재구매 고객: £2,915.68   
- 일회성 고객: £412.80   


- 재구매까지 평균 207일 (약 7개월)   
- 역설적 발견: 첫 구매 금액 낮을수록 재구매율 상승   

- 재구매 고객 첫 구매: £37.09   
- 일회성 고객 첫 구매: £88.70   

**결론: 부담 없는 첫 경험이 재구매로 이어진다.**